In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import numpy as np
import pandas as pd

In [11]:
#Read the tsv file into a dataframe
data =pd.read_csv("/content/drive/MyDrive/dataset_ner_tsv/merged_file.tsv",sep = '\t')
data

,text,label
0,COVID-19,S-Disease
1,COVID-19,S-Disease
2,COVID-19,S-Disease
3,malaria,S-Disease
4,HIV,S-Disease
...,...,...
113149,do,O
113150,Estado,O
113151,do,O
113152,Amazonas,O


In [12]:
data.dropna(inplace=True)

In [13]:

data.isnull().sum()

text     0
label    0
dtype: int64

In [4]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2023-03-20 09:29:16--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-03-20 09:29:17--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-03-20 09:29:17--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [5]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [7]:
embedding_path = '/content/glove.6B.100d.txt'
import numpy as np

embeddings_index = {}
with open(embedding_path) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [14]:
def text_to_vector(text):
    words = text.split()
    vectors = [embeddings_index.get(word, np.zeros(100)) for word in words]
    vector = np.mean(vectors, axis=0)
    return vector

X = np.array([text_to_vector(text) for text in data['text']])
y = np.array(data['label'])

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## LOGISTIC REGRESSION + GLOVE

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(random_state=0, max_iter=1000, multi_class='ovr')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9365914011753789


## NAIVE BAYES + GLOVE

In [37]:
from sklearn.preprocessing import MinMaxScaler #fixed import

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [38]:
# Train a Naive Bayes classifier
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_test)
nb_acc = accuracy_score(y_test, nb_pred)
print(f"Naive Bayes accuracy: " ,nb_acc)

Naive Bayes accuracy:  0.9312005655958641


## SVM + GLOVE

In [17]:
# Create SVM classifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
clf = SVC(kernel='rbf', C=1, decision_function_shape='ovr')
clf.fit(X_train, y_train)
# Predict the labels of the test data
y_pred = clf.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

In [18]:
print("Accuracy:", accuracy)

Accuracy: 0.9388007600194423


## RANDOM FOREST + GLOVE

In [20]:
from sklearn.ensemble import RandomForestClassifier

# Create Random Forest classifier
cl = RandomForestClassifier(n_estimators=100, random_state=0)

# Train the classifier on the training data
cl.fit(X_train, y_train)

# Predict the labels of the test data
y_pred = cl.predict(X_test)

In [21]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9392426317882551
